#  **Introducción**

<font size = 4>
El presente notebook, recoge el procedimiento seguido para la creación del modelo y su posterior fase de entrenamiento, validación y test.  
<br> <br>
 
La base de datos utilizada para la creación del clasificador de imágenes se denomina **MNIST**. 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers, losses
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization, Dropout, Conv2D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

<font size = 4>**Creacion del conjunto de entrenamiento y test** </font>

In [ ]:
test = pd.read_csv('../input/digit-recognizer/test.csv')
train = pd.read_csv('../input/digit-recognizer/train.csv')

lb = "label"
sc = 255.0
numClass = 10
imageWidht = 28
imageHeight = 28
imageChannels = 1
   
Y_train = train[lb]
X_train = train.drop(labels = lb,axis = 1)

X_train = X_train / sc

X_test = test / sc

Y_train = to_categorical(Y_train, num_classes = numClass)


X_train = X_train.values.reshape(-1,imageWidht,imageHeight,imageChannels)
X_test = X_test.values.reshape(-1,imageWidht,imageHeight,imageChannels)

<font size = 4>**Visualización de las imagenes del dataset** </font>

In [ ]:
import matplotlib.pyplot as plt
# PREVIEW IMAGES
plt.figure(figsize=(15,4.5))
for i in range(30):  
    plt.subplot(3, 10, i+1)
    plt.imshow(X_train[i].reshape((imageWidht,imageHeight)),cmap=plt.cm.binary)
    plt.axis('off')
plt.subplots_adjust(wspace=-0.1, hspace=-0.1)
plt.show()

<font size = 4>**Creacion del modelo** </font>

In [ ]:
#MY CODE
input_shape = (imageWidht,imageHeight,imageChannels)

myModel = keras.Sequential(
    [
        layers.Conv2D(32, kernel_size = 3, activation="relu", input_shape = input_shape),
        layers.BatchNormalization(),
        layers.Conv2D(32, kernel_size = 3, activation="relu"),
        layers.BatchNormalization(),
        layers.Conv2D(32, kernel_size = 5, strides=2, padding='same', activation="relu"),
        layers.Dropout(0.3),
        layers.Conv2D(64, kernel_size = 3, activation="relu"),
        layers.BatchNormalization(),
        layers.Conv2D(64, kernel_size = 3, activation="relu"),
        layers.BatchNormalization(),
        layers.Conv2D(64, kernel_size = 5, strides=2, padding='same', activation="relu"),
        layers.Dropout(0.3),
        layers.Conv2D(128, kernel_size = 4, activation="relu"),
        layers.BatchNormalization(),
        layers.Flatten(),
        layers.Dropout(0.3),
        layers.Dense(10, activation="softmax"),
    ]
   )
    
myModel.compile( optimizer='adam', loss="categorical_crossentropy", metrics=["accuracy"])

myModel.summary()

<font size = 4>**Entrenamiento** </font>

In [ ]:
early_stopping = EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=5, # how many epochs to wait before stopping
    restore_best_weights=True,)

epochs = 50

X_train2, X_val2, Y_train2, Y_val2 = train_test_split(X_train, Y_train, test_size = 0.1)

history = myModel.fit(
    X_train2, Y_train2,
    validation_data=(X_val2, Y_val2),
    batch_size=64,
    epochs=epochs,
    steps_per_epoch = X_train2.shape[0]//64,
    callbacks=[early_stopping],
    verbose=0, # suppress output since we'll plot the curves
)
history_df = pd.DataFrame(history.history)
history_df.loc[0:, ['loss', 'val_loss']].plot()
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()))

print(max(history.history['accuracy']))
print(max(history.history['val_accuracy']))

<font size = 4>**Almacenamiento del modelo entrenado** </font>

In [ ]:
myModel.save('myModelMNIST.h5')

<font size = 4>**Carga del modelo entrenado** </font>

In [ ]:
from keras.models import load_model
myModel = load_model('myModelMNIST.h5')

<font size = 4>**Predicción** </font>

In [ ]:
y_pred = myModel.predict(X_test)
X_test__ = X_test.reshape(X_test.shape[0], 28, 28)

fig, axis = plt.subplots(4, 4, figsize=(12, 14))
for i, ax in enumerate(axis.flat):
    ax.imshow(X_test__[i], cmap='binary')
    ax.set(title = f"Predict Number is {y_pred[i].argmax()}");